#**Externally Used Functions**

In [ ]:
ListOfActions = ["l", "r", "u", "d"]
InverseActionDict = {"l":"r", "r":"l", "u":"d", "d":"u"}

def performActionsOnPlainList(beginningState, listOfActions):
  for act in listOfActions:
    beginningState = performActionOnPlainList(beginningState, act)

  return beginningState

def invertActions(listOfActions):
  retList = []
  for act in listOfActions:
    retList.append(InverseActionDict.get(act))
  retList.reverse()
  return ''.join(retList)


def performActionOnPlainList(state, action):
  if not isValidMove(state, action):
      print(action, "Not Valid")
      return state
  openSpaceIndex = state.index('0')
  state = list(state);
  if action == "u":
      return swap(openSpaceIndex, openSpaceIndex - BoardSize, state)
  elif action == "d":
      return swap(openSpaceIndex, openSpaceIndex + BoardSize, state)
  elif action == "l":
     return swap(openSpaceIndex, openSpaceIndex - 1, state)
  elif action == "r":
        return swap(openSpaceIndex, openSpaceIndex + 1, state)

def isValidMove(state, action):
  openSpaceIndex = state.index('0')
  if action == "u":
       return openSpaceIndex > BoardSize - 1
  elif action == "d":
      return openSpaceIndex//BoardSize < BoardSize - 1 
  elif action == "l":
      return openSpaceIndex%BoardSize > 0
  elif action == "r":
      return openSpaceIndex%BoardSize < BoardSize - 1

def swap(openIndex, moveToIndex, list):
  temp = list[moveToIndex]
  list[moveToIndex] = list[openIndex]
  list[openIndex] = temp
  return ''.join(list)

def printNodeVals(node, nodesExpanded, functionName, newPS):
  nodePath = [node.action]
  tempNode = node.parent
  while tempNode != None and tempNode.parent != None:
    nodePath.append(tempNode.action)
    tempNode = tempNode.parent
    
  nodePath.reverse()

  print(f'{"Function Name: ":22}', functionName)
  if node.state != "failure":
    print(f'{"Final State: ":22}', ''.join(node.state))
    print(f'{"Action Set: ":22}', ''.join(nodePath))
    print(f'{"Path Cost: ":22}', len(nodePath))
    print(f'{"Nodes Expanded: ":22}', nodesExpanded)
    print(f'{"Original State: ":22}', newPS)
    finalState = performActionsOnPlainList(newPS, nodePath)
    print(f'{"Verification of path: ":22}', finalState)
  else:
    print(f'{"Final State: ":22}', ''.join(node.state))
    print(f'{"Nodes Expanded: ":22}', nodesExpanded)
  print()

#**Start of Notes**

Agents can follow this four phase problem solving process:

**Goal Formulation**: The agent adopts the the *goal* of reaching the solution. Goals organize behavior by limiting the objective and hence the actions to be considered.

**Problem Formulation**: the agent devises a description of states and actions necessary to reach the goal - an abstract model of the relevant part of the world. For our agent, one good model is to consider the actions of moving the open space a certain way, and therefore the only fact about the state of the problem that will change is due to an action at the current state (Think FSMs)

**Search**: Before taking any action in the real world, the agent simulates sequences of actions in its model, searching until it finds a sequence of actions that reaches the goal. Such a sequence is a solution. The agent might have to simulate multiple sequences that do not reach the goal, but eventually it will find a solution, or it will find that no solution is possible.

**Execution** The agent can now execute the actions in the solution, one at a time.

It is an important property that in a fully observable, deterministic, known environment, the solution to any problem is a fixed sequence of actions.

A **Search** problem can be defined as follows:

- A Set of possible states that the environment can be in. We call this the state space.
- The initial state that the agent starts in
-A set of one or more goals states. Sometimes there is one goal state, sometimes there is a small set of alternative goal states, and sometimes the goal is defined by a property that applies to many states (potentially an infinite number). We can account for all three of these properties by specifying an Is-Goal method for the problem.
-The actions available to the target. Given a state *s*, Action(*s*) returns a finite set of actions that can be executed in s. We say that each of these actions is **applicable** in *s*.
-A transition model, which describes what each action does. Result(*s*,*a*) retunrs the state that results from doing action in state s. 
-A action cost function, denoted by Action-Cost(*s*,*a*,*s'*) when we are programming of c(*s*,*a*,*s'*) when we are doing math, that gives the numeric cost of applying action *a* in state *s* to reach state *s'*. A problem solving agent should use a cost function that reflects its own performance measure.



In [ ]:
import queue
class Node():
    
    
  def __init__(self, state, parent, action, path_cost, zeroIndex):
      self.state = state
      self.parent = parent
      self.action = action
      self.path_cost = path_cost
      self.zeroIndex = zeroIndex
    
  def __lt__(self, other):
      return self.path_cost < other.path_cost if isinstance(other, Node) else 0# if not self or not other else 1
    
  def __eq__(self, other):
      return self.path_cost == other.path_cost if isinstance(other, Node) else 0 #if not self or not other else 0
    
  def __gt__(self, other):
      return self.path_cost > other.path_cost if isinstance(other, Node) else 0# if not self or not other else -1
       
class Problem():
    
    BoardSize = 4
    
    GoalState3x3 = list("123456780")
    GoalState4x4 = list("123456789ABCDEF0")
    
    GoalState = (GoalState3x3, GoalState4x4)[BoardSize == 4]

    ListOfActions = ["l", "r", "u", "d"]
    InverseActionDict = {"l":"r", "r":"l", "u":"d", "d":"u"}
    
    def __init__(self, initial, goal, boardSize):
        self.initial = initial[:]
        self.zeroIndex = initial.index("0")
        self.boardSize = boardSize
        self.goalState = goal[:]
        
    def invertActions(listOfActions):
        retList = []
        for act in listOfActions:
          retList.append(Problem.InverseActionDict.get(act))
        retList.reverse()
        return retList
    
    def isGoal(self, state):
      return ''.join(state) == ''.join(self.goalState)
        
    def action_cost(self, state, action, newstate):
      return 1
        
    def getMostRecentZeroIndex(self):
      return self.zeroIndex
     
    def performActions(self, beginningState, listOfActions):
      for act in listOfActions:
        beginningState = self.performAction(beginningState, act, beginningState.index("0"))
      return beginningState
   
    def performAction(self, state, action, zIndex):
      if not self.isValidMove(state, action, zIndex):
          return state
      if action == "u":
          return self.swap(zIndex, zIndex - self.boardSize, state)
      elif action == "d":
          return self.swap(zIndex, zIndex + self.boardSize, state)
      elif action == "l":
         return self.swap(zIndex, zIndex - 1, state)
      elif action == "r":
            return self.swap(zIndex, zIndex + 1, state)

    def isValidMove(self, state, action, zIndex):
      if action == "u":
           return zIndex >self.boardSize - 1
      elif action == "d":
          return zIndex//self.boardSize < self.boardSize - 1 
      elif action == "l":
          return zIndex%self.boardSize > 0
      elif action == "r":
          return zIndex%self.boardSize < self.boardSize - 1

    def swap(self, zIndex, moveToIndex, state):
      self.zeroIndex = moveToIndex
      temp = state[moveToIndex]
      state[moveToIndex] = state[zIndex]
      state[zIndex] = temp

      return state

#**Best First Search**
In pseudo-code, one of the first algorithms we look at is called 'Best-First-Search'. In this approach, we choose a node, *n*, with minimum value of some evaluation function, **f(*n*)**. On each iteration, we choose a node on the frontier with the minimum f(*n*) value, return if it its state is a goal state, and otherwise, apply Expand to generate child nodes. Each child node is added to the frontier if it has not been reached before, or is re-added if it is now being reached with a path that has a lower path cost than any previous path

In [ ]:
def best_first_search(problem):
  node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
  frontier = queue.PriorityQueue(0)
  frontier.put(node)
  reached = {''.join(problem.initial) : 0}
  nodesExpanded = 0
  while not frontier.empty():
    node = frontier.get()
    if problem.isGoal(node.state):
      return (node, nodesExpanded)
    nodesExpanded += 1
    for child in expand(problem, node):
        hashableState = ''.join(child.state)
        if not reached.get(hashableState) or child.path_cost < reached.get(hashableState):
            reached[hashableState] = child.path_cost
            frontier.put(child)
  return (Node("failure", None, None, None, None), nodesExpanded)
        
def expand(problem, node):
  retList = []
  origState = node.state[:]
  origZeroIndex = node.zeroIndex
  for act in problem.ListOfActions:
    # Dont make a node if its not a valid move
    if problem.isValidMove(node.state, act, node.zeroIndex):
      s = problem.performAction(node.state, act, node.zeroIndex)
      z = problem.getMostRecentZeroIndex()
      #reset 'last' zeroIndex to maintain correctness
      problem.zeroIndex = origZeroIndex
      #reset node state cause python
      node.state = origState[:]
      cost = node.path_cost + problem.action_cost(node.state, act, s)
      retList.append(Node(s, node, act, cost, z))
  return retList

#**Breadth First Search**

When all actions have the same cost, such as the sliding tile puzzle, another appropriate strategy would be 'Breadth First Search". In Breadth First Search, the **Root** node is expanded first, then all of the successors of the root are then expanded, and so on. This is a systematic strategy which is considered complete even in infinite spaces. As opposed to Best First Search, Breadth First Search uses a FIFO queue, which improves efficiency as it doens't have to sort for every new Node. Naturally, new Nodes will be deeper than their parents and therefore be put in the back of the queue. Additionally, 'reached' will no longer need to be a mapping, since with this approach 'once we've reached a node, we will find no quicker route' to it. 

Analyzing this a bit further, this means we can do an **early goal test**, i.e. we check to see if the node is a solution as soon as it is generated.

In [ ]:
def breadth_first_search(problem):
  node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
  frontier = queue.Queue(0)
  frontier.put(node)
  reached = {''.join(problem.initial) : 0}
  nodesExpanded = 0
  while not frontier.empty():
      node = frontier.get();
      nodesExpanded += 1
      for child in expand(problem, node):
          hashableState = ''.join(child.state)
          if problem.isGoal(child.state): #Early Goal Test, Done 'as soon' as node is generated
            return (child, nodesExpanded)
          if not reached.get(hashableState):
              reached[hashableState] = child.path_cost
              frontier.put(child)
  return (Node("failure", None, None, None, None), nodesExpanded)

#**Depth First Search**


In [ ]:
def depth_first_search(problem):
   node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
   frontier = queue.LifoQueue(0) #stack
   frontier.put(node)
   nodesExpanded = 0
   while not frontier.empty():
     node = frontier.get()
     if problem.isGoal(node.state): #Late Goal Test, Done after parent has been expanded
       return (node, nodesExpanded)
     elif node.path_cost + 1 > 30:
       continue
     elif not is_cycle(node):
       nodesExpanded += 1
       for child in expand(problem, node):
         frontier.put(child)

  
def is_cycle(node):
  nodePath = ''.join(node.state)
  node2 = node.parent
  while node2 != None:
    if nodePath == ''.join(node2.state):
      return True
    node2 = node2.parent
  return False

#**Depth Limited and Iterative Deepening Search**

Depth First Searches can be more efficient on memory than Best First and Breadth First searches, however in an infinite plane, DFS might never stop. This can arguably cause it to be 'incomplete'. One approach to using DFS while making it 'complete' is to add a limiter to it. That is, once the search has reached a certain depth, it must return up path and continue in another direction. To expand on this even further, we can do it iteratively. This means that if a solution is not found at a certain depth, we can iterate to the next depth and try again. This does cause some concern because it will cause nodes to be repeatedly generated with every new iteration. However, since these nodes are closer to the root, generally, this additional overhead can be ignored somewhat. Just like with Breadth First Search, we can do an **early goal test**

With this approach, we also do not have to keep all the explored nodes in memory, but it would do us well to check for repeated cycles to ensure we don't end up in an infinite loop.


In [ ]:
def iterative_deepening_search(problem, limit):
  for i in range(1, limit):
    resultNode = depth_limited_search(problem, i)
    if not resultNode == i:
      return resultNode
  
  return Node("failure", None, None, None, None)

def depth_limited_search(problem, limit):
   node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
   frontier = queue.LifoQueue(0) #stack
   frontier.put(node)
   nodesExpanded = 0
   while not frontier.empty():
     node = frontier.get()
     if problem.isGoal(node.state): 
       return (node, nodesExpanded)
     elif node.path_cost + 1 > limit:
       continue
     elif not is_cycle(node):
       nodesExpanded += 1
       for child in expand(problem, node):
         frontier.put(child)
  
   return limit

#**Global Test Values**

In [ ]:
BoardSize = 3
    
GoalState3x3 = list("123456780")
GoalState4x4 = list("123456789ABCDEF0")
    
GoalState = (GoalState3x3, GoalState4x4)[BoardSize == 4]

problemSet = (list("123456780"), list("123456789ABCDEF0"))[BoardSize == 4]
moveSet = ("uuldrdludluurrdlu","udlrudlulrllurudlrulr" )[BoardSize == 4]
newProblemSet = performActionsOnPlainList(problemSet, moveSet)


#**Uninformed Search Tests**

In [ ]:
problem = Problem(list("123406785"), GoalState,  BoardSize)

nodeList = [
            (breadth_first_search(problem), "Breadth First"), 
            ]

for ((node, expandedNodes), functionName) in nodeList:
  printNodeVals(node, expandedNodes, functionName, '' + newProblemSet)

Original State:        506123478

Function Name:         Breadth First
Final State:           failure
Nodes Expanded:        181441



#**Informed Search Strategies**

Previously, we preformed search using an un-informed approach. In this sense, we were un-informed with how close or far we were from the goal state, and only worried about ordering our nodes based on path cost. With an Informed approach, we use hints about the location of our goals to aid our search. These 'hints' come in the form of a **heuristic function**. 

For our sliding tile puzzle, these would be like the number of tiles out of place, or the 'Manhattan Distance' each tile is away from is goal position.



#**Heursitic Functions**

In [ ]:
def getOutOfPlace(state, goalState, boardSize):
      retVal = 0
      for s in goalState:
        if goalState.index(s) != state.index(s) and s != '0':
          retVal += 1
        else:
          retVal += 0
  
      return retVal
      
def getManhattanDis(state, goalState, boardSize):
      retVal = 0
      for s in state:
          if s != '0': 
              retVal += getManhattanForElement(s, state.index(s), goalState, boardSize)
      return retVal

def getManhattanForElement(element, index, goalState, boardSize):
  return rowOffset(element, index, goalState, boardSize) + columnOffset(element, index, goalState, boardSize)

def rowOffset(element, index, goalState, boardSize):
    return abs(getConversion(element)//boardSize - 
                index//boardSize)
  
def columnOffset(element, index, goalState, boardSize):
    return abs(index % boardSize - getConversion(element)%boardSize)
  
def getConversion(element):
  if 64 < ord(element) < 71: 
    return 14 - abs(int(element, 16) - int("F", 16))
  else:
     return int(element) - 1

# Adds Distance to Zero weight to Manhattan Distance, this will most likely over estimate solution, 
# But may add a better hint as to right direction, possibly resulting in less nodes expanded
def getWeightedSum(state, goalState, boardSize):
  zeroIndex = state.index('0')
  retVal = 0
  for element in state:
    if element != '0':
      elIndex = state.index(element)
      retVal += distanceToZero(elIndex, zeroIndex, boardSize) + getManhattanForElement(element, elIndex, goalState, boardSize)
  return retVal

def getWieghtedAvg(state, goalState, boardSize):
  zeroIndex = state.index('0')
  retVal = 0
  for element in state:
    if element != '0':
      elIndex = state.index(element)
      sum = distanceToZero(elIndex, zeroIndex, boardSize) + getManhattanForElement(element, elIndex, goalState, boardSize)
      retVal += sum // 2
  return retVal
  
def getDistanceToZero(state, goalState, boardSize):
  zeroIndex = state.index('0')
  retVal = 0
  for element in state:
    retVal += distanceToZero(state.index(element), zeroIndex, boardSize)
  return retVal

def distanceToZero(elIndex, zeroIndex, boardSize):
  return abs(elIndex%boardSize - zeroIndex%boardSize) + abs(elIndex//boardSize - zeroIndex//boardSize)

def getWeightedProduct(state, goalState, boardSize):
  zeroIndex = state.index('0')
  retVal = 0
  for element in state:
    elIn = state.index(element)
    mhn = getManhattanForElement(element, elIn, goalState, boardSize)
    zds = distanceToZero(elIn, zeroIndex, boardSize)
    retVal += (mhn + zds) * mhn
  return retVal

def getProductOverSum(state, goalState, boardSize):
  zeroIndex = state.index('0')
  retVal = 0
  for element in state:
    elIn = state.index(element)
    mhn = getManhattanForElement(element, elIn, goalState, boardSize)
    zds = distanceToZero(elIn, zeroIndex, boardSize)
    product = ((mhn + zds) * mhn)
    sum = (distanceToZero(elIn, zeroIndex, boardSize) + getManhattanDis(state, goalState, boardSize))
    if sum != 0:
      retVal +=  product/sum
  return retVal

#**Greedy Best First Search**

**Greedy Best First** Search is a form of best first search that expands the first node with the lowest *h(n)* value - the node that appears to be the closest to the goal - on the grounds that this is likely to lead to a solution quickly. In this case, *f(n)* = *h(n)*.

To reflect this in our code, we use the heuristic of choice as our method to sort our entries into the priority queue.

In [ ]:
def best_first_search_w_heuristic(problem, heuristicFunction):
  node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
  frontier = queue.PriorityQueue(0)
  frontier.put((heuristicFunction(node.state, problem.initial[:], problem.boardSize),node)) # <-- Hueristic determines order
  reached = {''.join(problem.initial) : 0}
  nodesExpanded = 0
  while not frontier.empty():
    (hVal, node) = frontier.get()
    if problem.isGoal(node.state):
      return (node, nodesExpanded)
    nodesExpanded += 1
    if nodesExpanded > 1000000:
      break
    for child in expand(problem, node):
        hashableState = ''.join(child.state)
        if not reached.get(hashableState) or child.path_cost < reached.get(hashableState):
            reached[hashableState] = child.path_cost
            frontier.put((heuristicFunction(child.state, problem.initial[:], problem.boardSize), child)) # <-- Hueristic determines order
  return (Node("failure", None, None, None, None), nodesExpanded)


#**Informed Search Test**

In [ ]:
print(f'{"Original State: ":22}', newProblemSet)
print()
problem = Problem(list(newProblemSet), GoalState,  BoardSize)

nodeList = [(best_first_search_w_heuristic(problem, getManhattanDis), "Best First With Heuristic (Manhattan Distance)" ),
            (best_first_search_w_heuristic(problem, getOutOfPlace), "Best First With Heuristic (Out of Place)") 
           ]

for ((node, expandedNodes), functionName) in nodeList:
  printNodeVals(node, expandedNodes, functionName, '' + newProblemSet)

Original State:        1034627859ACDEBF



KeyboardInterrupt: ignored

#**A\* Search**

The next search algorithm works similarly to Best First w/ Heuristic, however it adds to the function it uses to determine the next node to expand. The previous algorithm used *h(n)* which was the heuristic ALONE. A* adds to that by keeping track of the path cost as well as the heursitic by using *f(n)* + *h(n)*. 

By doing so, we avoid from straying away from our end goal, and more effectively choose the best nodes to expand.

In [ ]:
def a_star_search(problem, heuristicFunction):
  node = Node(problem.initial[:], None, "", 0, problem.initial.index("0"))
  frontier = queue.PriorityQueue(0)
  frontier.put((heuristicFunction(node.state, problem.initial[:], problem.boardSize) + node.path_cost,node)) # <-- Hueristic determines order
  reached = {''.join(problem.initial) : 0}
  nodesExpanded = 0
  while not frontier.empty():
    (hVal, node) = frontier.get()
    if problem.isGoal(node.state):
      return (node, nodesExpanded)
    nodesExpanded += 1
    for child in expand(problem, node):
        hashableState = ''.join(child.state)
        if not reached.get(hashableState) or child.path_cost < reached.get(hashableState):
            reached[hashableState] = child.path_cost
            frontier.put((heuristicFunction(child.state, problem.initial[:], problem.boardSize) + child.path_cost, child)) # <-- Hueristic determines order
  return (Node("failure", None, None, None, None), nodesExpanded)


#**A\* Search Test**

In [ ]:
print(f'{"Original State: ":22}', newProblemSet)
print()

problem = Problem(list(newProblemSet), GoalState,  BoardSize)

nodeList = [(a_star_search(problem, getManhattanDis), "A* With Heuristic (Manhattan Distance)" ),
            (a_star_search(problem, getOutOfPlace),"A* With Heuristic (Out of Place)") 
           ]

for ((node, expandedNodes), functionName) in nodeList:
  printNodeVals(node, expandedNodes, functionName, '' + newProblemSet)

Original State:        1034627859ACDEBF

Function Name:         A* With Heuristic (Manhattan Distance)
Final State:           123456789ABCDEF0
Action Set:            dldrrdr
Path Cost:             7
Nodes Expanded:        7
Original State:        1034627859ACDEBF
Verification of path:  123456789ABCDEF0

Function Name:         A* With Heuristic (Out of Place)
Final State:           123456789ABCDEF0
Action Set:            dldrrdr
Path Cost:             7
Nodes Expanded:        630
Original State:        1034627859ACDEBF
Verification of path:  123456789ABCDEF0



#**A\* Cont.**

A* is complete. Whether it is cost-optimal or not is another thing. This property depends on a few things, but a key property is the heurisitic. At least, the heuristics ability to *never overestimate* the cost to reach the goal. If it is the case that it never over esitmates the cost, it is said to be **Admissible** (not to be confused with something being admissible in court). 

For example, let's say our cost to goal is approx 28 moves. Then the heuristic function at each step should be able to estimate the cost, respectively, for each step towards the goal without going over the cost from that step, i.e. both the heuristic and remaining path cost go down as you near the goal.

Another factor is called **Consistency**. A heuristic is consistent if, for every node *n* and every child node *n'* generated by an action *a*, we have:

*h(n)* <= cost_of_action(*n*, *a*, *n'*) + *h(n')*. 

This means that, if your new heuristic value, added to your cost value is less than your old hueristic, then something is wrong. Your hueristic essentially *points* directly to the goal, so if you do an action and add the next hueristic and it is less than or equal to the old hueristic you found a *magic path* that shouldn't exsist. If this happens, your hueristic is pointing in the wrong direction.

Every **consistent** heuristic is **admissible**, but not every **admissible** hueristic is **consistent**.

#**Assignment Problems 3x3**

In [ ]:
listOf3x3Problems = ["160273485","462301587","821574360","840156372","530478126","068314257","453207186","128307645","035684712","684317025","028514637","618273540","042385671",
"420385716","054672813","314572680" ,"637218045" ,"430621875" ,"158274036" ,"130458726"]

BoardSize = 3
    
GoalState3x3 = list("123456780")
GoalState4x4 = list("123456789ABCDEF0")
       
GoalState = (GoalState3x3, GoalState4x4)[BoardSize == 4]

for problemSet in listOf3x3Problems:
  problem = Problem(list(problemSet), GoalState,  BoardSize)

  nodeList = [(best_first_search(problem), "Best First w Path Cost"),  
              (breadth_first_search(problem), "Breadth First"), 
              (depth_first_search(problem), "Depth First"), 
              (iterative_deepening_search(problem, 30), "Iterative Deepening Depth First"),
              (best_first_search_w_heuristic(problem, getManhattanDis), "Best First With Heuristic (Manhattan Distance)" ),
              (best_first_search_w_heuristic(problem, getOutOfPlace), "Best First With Heuristic (Out of Place)") ,
              #(best_first_search_w_heuristic(problem, getWeightedSum), "Best First With Heuristic (Weighted Sum)"),
              #(best_first_search_w_heuristic(problem, getWeightedProduct), "Best First With Heuristic (Weighted Product)"),
              #(best_first_search_w_heuristic(problem, getProductOverSum), "Best First With Heuristic (Weighted Product/Sum)"),
              (a_star_search(problem, getManhattanDis), "A* With Heuristic (Manhattan Distance)" ),
              (a_star_search(problem, getOutOfPlace),"A* With Heuristic (Out of Place)"),
              #(a_star_search(problem, getWeightedSum), "A* With Heuristic (Weighted Sum)"),
              #(a_star_search(problem, getWeightedProduct), "A* With Heuristic (Weighted Product)"),
              #(a_star_search(problem, getProductOverSum), "A* With Heuristic (Weighted Product/Sum)")  
            ]

  for ((node, expandedNodes), functionName) in nodeList:
    printNodeVals(node, expandedNodes, functionName, '' + problemSet)


Function Name:         Best First w Path Cost
Final State:           123456780
Action Set:            ddluurdlldrruuldrd
Path Cost:             18
Nodes Expanded:        26616
Original State:        160273485
Verification of path:  123456780

Function Name:         Breadth First
Final State:           123456780
Action Set:            ddluurdlldrruuldrd
Path Cost:             18
Nodes Expanded:        17318
Original State:        160273485
Verification of path:  123456780

Function Name:         Depth First
Final State:           123456780
Action Set:            ddluurddlurdlurdluldrruuldrd
Path Cost:             28
Nodes Expanded:        43577
Original State:        160273485
Verification of path:  123456780

Function Name:         Iterative Deepening Depth First
Final State:           123456780
Action Set:            ddluurdlldrruuldrd
Path Cost:             18
Nodes Expanded:        631
Original State:        160273485
Verification of path:  123456780

Function Name:         Best Fir

KeyboardInterrupt: ignored

Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            ddluurdlldrruuldrd  
Path Cost:             18  
Nodes Expanded:        26616  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            ddluurdlldrruuldrd  
Path Cost:             18  
Nodes Expanded:        17318  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluurddlurdlurdluldrruuldrd  
Path Cost:             28  
Nodes Expanded:        43577  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            ddluurdlldrruuldrd  
Path Cost:             18  
Nodes Expanded:        631  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ddluldrruuldrdlurulddrulurdd  
Path Cost:             28  
Nodes Expanded:        118  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            ldrdluurdlldrruldlururdldrulldruurdd  
Path Cost:             36  
Nodes Expanded:        39879  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ddluurdlldrruuldrd  
Path Cost:             18  
Nodes Expanded:        427  
Original State:        160273485  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            ddluurdlldrruuldrd  
Path Cost:             18  
Nodes Expanded:        17857  
Original State:        160273485  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rdlurulldrdluruldrrulddr  
Path Cost:             24  
Nodes Expanded:        144587  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rdlurulldrdluruldrrulddr  
Path Cost:             24  
Nodes Expanded:        119026  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            druldruullddruldruuldrrulddr  
Path Cost:             28  
Nodes Expanded:        1670509  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            rdlurulldrdluruldrrulddr  
Path Cost:             24  
Nodes Expanded:        761848  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            urdlldrrullurrdldlurrdlluruldrdruulldrurdd  
Path Cost:             42  
Nodes Expanded:        477  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drulldrrulurdldruuldlurrdd  
Path Cost:             26  
Nodes Expanded:        152842  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rdlurulldrdluruldrrulddr  
Path Cost:             24  
Nodes Expanded:        2843  
Original State:        462301587  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rdlurulldrdluruldrrulddr  
Path Cost:             24  
Nodes Expanded:        136900  
Original State:        462301587  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            lurulldrrdllurdrululddruurdd  
Path Cost:             28  
Nodes Expanded:        176787  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            lulurdldruruldlurdruldlurdrd  
Path Cost:             28  
Nodes Expanded:        173419  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uldrululddruurdllurddruuldldrr  
Path Cost:             30  
Nodes Expanded:        5636490  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            luurdllurdldrulurrdluldrurdd  
Path Cost:             28  
Nodes Expanded:        5110762  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            luldrulurdrulldrrulldruldrurddllurdruldlurrdluldrr  
Path Cost:             50  
Nodes Expanded:        330  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            lluurddrullurrdllurddruldlurdr  
Path Cost:             30  
Nodes Expanded:        171658  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            luruldrdlluurddrulldrulurrdd  
Path Cost:             28  
Nodes Expanded:        8564  
Original State:        821574360  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            luruldlurdruldldrulurdruldrd  
Path Cost:             28  
Nodes Expanded:        177067  
Original State:        821574360  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            dllurdldrurdlulurdrulddr  
Path Cost:             24  
Nodes Expanded:        117159  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            dllurdldrrullurddruulddr  
Path Cost:             24  
Nodes Expanded:        109030  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluulddruurdlldrruuldruldldrr  
Path Cost:             30  
Nodes Expanded:        530140  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            dllurdldrurdlulurdrulddr  
Path Cost:             24  
Nodes Expanded:        488979  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            llddruurddluruldlurrdluldrrdlurulddrulurdd  
Path Cost:             42  
Nodes Expanded:        116  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dllurdldrulurdrdllurruldldrr  
Path Cost:             28  
Nodes Expanded:        106588  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            dllurdldrulurdrdlurulddr  
Path Cost:             24  
Nodes Expanded:        1272  
Original State:        840156372  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dllurdldrulurdrdlurulddr  
Path Cost:             24  
Nodes Expanded:        109437  
Original State:        840156372  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            dldlurdruulldrdrululddrr  
Path Cost:             24  
Nodes Expanded:        129328  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            dldlurdruulldrdrululddrr  
Path Cost:             24  
Nodes Expanded:        112698  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluuldrrullddruuldrrdluruldrd  
Path Cost:             30  
Nodes Expanded:        1008020  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            dldlurdruulddrullurdldrr  
Path Cost:             24  
Nodes Expanded:        250736  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            lldrrdluldruulddrruldluurddlurrdlluurddlurulddrr  
Path Cost:             48  
Nodes Expanded:        376  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            lldrdluurddlurrullddruurdd  
Path Cost:             26  
Nodes Expanded:        143882  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            dldlurdruuldlurddrulldrr  
Path Cost:             24  
Nodes Expanded:        2588  
Original State:        530478126  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dldlurdruulddrullurdldrr  
Path Cost:             24  
Nodes Expanded:        126011  
Original State:        530478126  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            drrulldrdlurdruuldldrr  
Path Cost:             22  
Nodes Expanded:        80955  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            drrulldrdlurdruuldldrr  
Path Cost:             22  
Nodes Expanded:        64865  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddruurdlurdlulddruldrruullddrr  
Path Cost:             30  
Nodes Expanded:        322360  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            drrullddruldrruuldldrr  
Path Cost:             22  
Nodes Expanded:        168557  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            drrulldrrulddlurdrulldruurddluurddlurulddrulurdd  
Path Cost:             48  
Nodes Expanded:        158  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drrulldrdlurdruuldldrr  
Path Cost:             22  
Nodes Expanded:        277700  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            drrulldrdlurdruuldldrr  
Path Cost:             22  
Nodes Expanded:        370  
Original State:        068314257  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drrullddruldrruuldldrr  
Path Cost:             22  
Nodes Expanded:        94188  
Original State:        068314257  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rdluulddruuldrdr  
Path Cost:             16  
Nodes Expanded:        13605  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rdluulddruuldrdr  
Path Cost:             16  
Nodes Expanded:        7637  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            druuldrulddruullddruuldrdruldr  
Path Cost:             30  
Nodes Expanded:        520442  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            rdluulddruuldrdr  
Path Cost:             16  
Nodes Expanded:        8942  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            lurdrdluldrruulldrurdd  
Path Cost:             22  
Nodes Expanded:        225  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            urdldruuldrdluulddruuldrrd  
Path Cost:             26  
Nodes Expanded:        31574  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rdluulddruuldrdr  
Path Cost:             16  
Nodes Expanded:        173  
Original State:        453207186  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rdluulddruuldrdr  
Path Cost:             16  
Nodes Expanded:        10061  
Original State:        453207186  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            lurdrulldrdlurrdlurd  
Path Cost:             20  
Nodes Expanded:        54146  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            lurdrulldrdlurrdlurd  
Path Cost:             20  
Nodes Expanded:        33677  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            druulddrulldrrulldruurddllurrd  
Path Cost:             30  
Nodes Expanded:        151617  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            lurdrullddruldrruldr  
Path Cost:             20  
Nodes Expanded:        143415  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ldrruldrulldrurdluurdlurddluurddlurulddrulurdd  
Path Cost:             46  
Nodes Expanded:        222  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            lurdrullddruldrruldr  
Path Cost:             20  
Nodes Expanded:        16775  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            lurdrulldrdlurdruldr  
Path Cost:             20  
Nodes Expanded:        325  
Original State:        128307645  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            lurdrullddruldrruldr  
Path Cost:             20  
Nodes Expanded:        44132  
Original State:        128307645  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rdldrurdluldruulddruruldrd  
Path Cost:             26  
Nodes Expanded:        166113  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rdldrurdluldruulddruruldrd  
Path Cost:             26  
Nodes Expanded:        147304  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddruuldrulddruuldrrdllurruldrd  
Path Cost:             30  
Nodes Expanded:        862016  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            rddluruldrdlurrdllurruldrd  
Path Cost:             26  
Nodes Expanded:        1846814  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rrddluurdluldrrdlurulldrruldlurdrulddruuldrdllurulddrr  
Path Cost:             54  
Nodes Expanded:        452  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rddluruldrdlurrdllurruldrd  
Path Cost:             26  
Nodes Expanded:        145258  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rdldrurdluldruulddruruldrd  
Path Cost:             26  
Nodes Expanded:        3665  
Original State:        035684712  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rdldrurdluldruulddruruldrd  
Path Cost:             26  
Nodes Expanded:        157799  
Original State:        035684712  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rurullddrurullddrruulldrdr  
Path Cost:             26  
Nodes Expanded:        163566  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rurulldrdlurrulldrdruulddr  
Path Cost:             26  
Nodes Expanded:        147823  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uurddluurddluurdrulldrrdllurrd  
Path Cost:             30  
Nodes Expanded:        534671  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            rurullddrurullddrruulldrdr  
Path Cost:             26  
Nodes Expanded:        2187587  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rruulldrrulldrrulddlurdrulldruurddluurddlurulddrulurdd  
Path Cost:             54  
Nodes Expanded:        165  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rurullddrurullddrruulldrdr  
Path Cost:             26  
Nodes Expanded:        320132  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rurullddrurullddrruulldrdr  
Path Cost:             26  
Nodes Expanded:        1375  
Original State:        684317025  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rurullddrurullddrruulldrdr  
Path Cost:             26  
Nodes Expanded:        168486  
Original State:        684317025  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            drrdlurulldrdlurrdluurdd  
Path Cost:             24  
Nodes Expanded:        137617  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            drrdlurulldrdlurrdluurdd  
Path Cost:             24  
Nodes Expanded:        110496  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddrurulddluurddrulldrulurrdldr  
Path Cost:             30  
Nodes Expanded:        1597406  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            drrdlurulldrdlurrdluurdd  
Path Cost:             24  
Nodes Expanded:        428113  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rddrulldrulurrdldrululddrurdlluruldrdruulldrurdd  
Path Cost:             48  
Nodes Expanded:        445  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rddluurdrdluldrruulddrulldrr  
Path Cost:             28  
Nodes Expanded:        162460  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            drrdlurulldrdlurrdluurdd  
Path Cost:             24  
Nodes Expanded:        1034  
Original State:        028514637  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drrdlurulldrdlurrdluurdd  
Path Cost:             24  
Nodes Expanded:        122534  
Original State:        028514637  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            luuldrrulddruulldrdlurdr  
Path Cost:             24  
Nodes Expanded:        133625  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            luuldrrulddruulldrdlurdr  
Path Cost:             24  
Nodes Expanded:        106525  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uuldrdluurdldrullurdldrrulldrr  
Path Cost:             30  
Nodes Expanded:        1051757  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            luuldrrulddruullddruldrr  
Path Cost:             24  
Nodes Expanded:        628080  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            lulurdldrruuldrdluurdlldrruldlurrd  
Path Cost:             34  
Nodes Expanded:        135  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            luuldrrulddruullddruldrr  
Path Cost:             24  
Nodes Expanded:        304856  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            luuldrrulddruulldrdlurdr  
Path Cost:             24  
Nodes Expanded:        1659  
Original State:        618273540  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            luuldrrulddruullddruldrr  
Path Cost:             24  
Nodes Expanded:        135856  
Original State:        618273540  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rrdlldrrullurrdldlurulddrr  
Path Cost:             26  
Nodes Expanded:        164021  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rrdlldrrullurrdldlurulddrr  
Path Cost:             26  
Nodes Expanded:        141085  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddruuldrdruldlurulddrrulurdldr  
Path Cost:             30  
Nodes Expanded:        716030  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            ddrurdlulurdruldlurdrulddr  
Path Cost:             26  
Nodes Expanded:        149283  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ddruuldrrdlurulldrruldlurdrulddruuldrdllurulddrr  
Path Cost:             48  
Nodes Expanded:        448  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rrdlldrrullurrdldlurulddrr  
Path Cost:             26  
Nodes Expanded:        320905  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rrdlldrrullurrdldlurulddrr  
Path Cost:             26  
Nodes Expanded:        4034  
Original State:        042385671  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rrdlldrrullurrdldlurulddrr  
Path Cost:             26  
Nodes Expanded:        168031  
Original State:        042385671  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            dllurddlurrullddrurd  
Path Cost:             20  
Nodes Expanded:        51102  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            dllurddlurrullddrurd  
Path Cost:             20  
Nodes Expanded:        33413  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluurddluurdlurdllurddruuldrd  
Path Cost:             30  
Nodes Expanded:        12620  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            dllurddlurrullddrurd  
Path Cost:             20  
Nodes Expanded:        53186  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ddlulurrdlurddluurdldruulldrruldlurrdd  
Path Cost:             38  
Nodes Expanded:        168  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dllurdrulddlurulddrurd  
Path Cost:             22  
Nodes Expanded:        17658  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            dllurddlurrullddrurd  
Path Cost:             20  
Nodes Expanded:        553  
Original State:        420385716  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dllurddlurrullddrurd  
Path Cost:             20  
Nodes Expanded:        36462  
Original State:        420385716  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        74038  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        68398  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddruurdldlurdluurdrdlulurrdldr  
Path Cost:             30  
Nodes Expanded:        257572  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        88204  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rrddlulurrdlldrrulldrurdlluurddlurulddrr  
Path Cost:             40  
Nodes Expanded:        277  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        279438  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        531  
Original State:        054672813  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            drdlururddluuldrurdldr  
Path Cost:             22  
Nodes Expanded:        91649  
Original State:        054672813  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            luldruuldrurdluldrurdldr  
Path Cost:             24  
Nodes Expanded:        125858  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            luldruuldrurdluldrurdldr  
Path Cost:             24  
Nodes Expanded:        103848  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uulddluurrddlulurdruldrdllurdr  
Path Cost:             30  
Nodes Expanded:        640688  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            luldruuldrurdluldrurdldr  
Path Cost:             24  
Nodes Expanded:        760013  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            luldruruldlurrdldruulldrrulldrruldrdlluurdldrr  
Path Cost:             46  
Nodes Expanded:        309  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            uulddlurrdlluruldrurdldluurrdd  
Path Cost:             30  
Nodes Expanded:        171725  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            luldruuldrurdluldrurdldr  
Path Cost:             24  
Nodes Expanded:        1648  
Original State:        314572680  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            luldruuldrurdluldrurdldr  
Path Cost:             24  
Nodes Expanded:        124184  
Original State:        314572680  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            rruulldrrdluulddrruldr  
Path Cost:             22  
Nodes Expanded:        80696  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            rruulldrrdluulddrruldr  
Path Cost:             22  
Nodes Expanded:        57752  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uurdldruuldrurdlurddluulddrurd  
Path Cost:             30  
Nodes Expanded:        2143160  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            rruuldrdlulurdldrruldr  
Path Cost:             22  
Nodes Expanded:        295389  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rruulldrulddrruldlurdrulldrurd  
Path Cost:             30  
Nodes Expanded:        53  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rruuldrdlulurdldrruldr  
Path Cost:             22  
Nodes Expanded:        267788  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            rruuldrdlulurdldrruldr  
Path Cost:             22  
Nodes Expanded:        551  
Original State:        637218045  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            rruuldrdlulurdldrruldr  
Path Cost:             22  
Nodes Expanded:        99423  
Original State:        637218045  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            dlldruurdllurrddlurd  
Path Cost:             20  
Nodes Expanded:        49804  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            ldldruruldlurrddlurd  
Path Cost:             20  
Nodes Expanded:        30374  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluurddlurdluldrulurrddluurdd  
Path Cost:             30  
Nodes Expanded:        48135  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            dlurdlldrulurrddlurd  
Path Cost:             20  
Nodes Expanded:        37838  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ldldrruldrulldrurdlluurddlurulddrr  
Path Cost:             34  
Nodes Expanded:        222  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dlldruurdllurrddlurd  
Path Cost:             20  
Nodes Expanded:        250994  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ldruldldrulurrddlurd  
Path Cost:             20  
Nodes Expanded:        586  
Original State:        430621875  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dlldruurdllurrddlurd  
Path Cost:             20  
Nodes Expanded:        58811  
Original State:        430621875  
Verification of path:  123456780  
  
<hr>  
    
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        124275  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        113874  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            uurdlurddluurddruulddruldlurdr  
Path Cost:             30  
Nodes Expanded:        2621063  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        174642  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ruldrurulldrrdluuldrruldrdluurdldruulldrruldlurrdluldrrd  
Path Cost:             56  
Nodes Expanded:        444  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        124831  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        1808  
Original State:        158274036  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            urdruulddruldlurdruuldrd  
Path Cost:             24  
Nodes Expanded:        128816  
Original State:        158274036  
Verification of path:  123456780  
  
<hr>  
   
Function Name:         Best First w Path Cost  
Final State:           123456780  
Action Set:            dldruulddruldr  
Path Cost:             14  
Nodes Expanded:        3322  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         Breadth First  
Final State:           123456780  
Action Set:            dldruuldrdlurd  
Path Cost:             14  
Nodes Expanded:        2702  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         Depth First  
Final State:           123456780  
Action Set:            ddluurddluurddlurdluurdldr  
Path Cost:             26  
Nodes Expanded:        1500  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456780  
Action Set:            dldruulddruldr  
Path Cost:             14  
Nodes Expanded:        743  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            ldrdluurddlurulddr  
Path Cost:             18  
Nodes Expanded:        73  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            ddlurulddrulurdldruuldrd  
Path Cost:             24  
Nodes Expanded:        176  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456780  
Action Set:            dldruuldrdlurd  
Path Cost:             14  
Nodes Expanded:        110  
Original State:        130458726  
Verification of path:  123456780  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456780  
Action Set:            dldruuldrdlurd  
Path Cost:             14  
Nodes Expanded:        1731  
Original State:        130458726  
Verification of path:  123456780  

#**Assignment Problems 4x4 Easy**

In [ ]:
listOf4x4EProblems = ["16235A749C08DEBF", "0634217859ABDEFC", "012456379BC8DAEF", "51246A38097BDEFC", "12345678D9CFEBA0"]

BoardSize = 4
    
GoalState3x3 = list("123456780")
GoalState4x4 = list("123456789ABCDEF0")
       
GoalState = (GoalState3x3, GoalState4x4)[BoardSize == 4]

for problemSet in listOf4x4EProblems:
  problem = Problem(list(problemSet), GoalState,  BoardSize)

  nodeList1 = [(best_first_search(problem), "Best First w Path Cost"),  
              (breadth_first_search(problem), "Breadth First"), 
              (depth_first_search(problem), "Depth First"), 
              (iterative_deepening_search(problem, 30), "Iterative Deepening Depth First")
            ]

  for ((node, expandedNodes), functionName) in nodeList1:
    printNodeVals(node, expandedNodes, functionName, '' + problemSet)

  nodeList2 = [(best_first_search_w_heuristic(problem, getManhattanDis), "Best First With Heuristic (Manhattan Distance)" ),
              (best_first_search_w_heuristic(problem, getOutOfPlace), "Best First With Heuristic (Out of Place)") ,
              (a_star_search(problem, getManhattanDis), "A* With Heuristic (Manhattan Distance)" ),
              (a_star_search(problem, getOutOfPlace),"A* With Heuristic (Out of Place)"),
              #(a_star_search(problem, getWeightedSum), "A* With Heuristic (Weighted Sum)"),
              #(a_star_search(problem, getWeightedProduct), "A* With Heuristic (Weighted Product)"),
              #(a_star_search(problem, getWeightedAvg), "A* With Heuristic (Weighted Avg)")]
  for ((node, expandedNodes), functionName) in nodeList2:
    printNodeVals(node, expandedNodes, functionName, '' + problemSet)


NameError: ignored

Function Name:         Best First w Path Cost  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        4879  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Breadth First  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        1808  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        2570  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           failure  
Nodes Expanded:        1000001  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        12  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            luurrddldr  
Path Cost:             10  
Nodes Expanded:        12167  
Original State:        16235A749C08DEBF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First w Path Cost  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        2294  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Breadth First  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        1460  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        706  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           failure  
Nodes Expanded:        1000001  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        15  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            drulddrrrd  
Path Cost:             10  
Nodes Expanded:        3902  
Original State:        0634217859ABDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First w Path Cost  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        2696  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Breadth First  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        983  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        1688  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           failure  
Nodes Expanded:        1000001  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            rrdrdlldrr  
Path Cost:             10  
Nodes Expanded:        9898  
Original State:        012456379BC8DAEF  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First w Path Cost  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        2476  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Breadth First  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        1382  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        2006  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           failure  
Nodes Expanded:        1000001  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            rulurrddrd  
Path Cost:             10  
Nodes Expanded:        14142  
Original State:        51246A38097BDEFC  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First w Path Cost  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        2532  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Breadth First  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        1577  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Iterative Deepening Depth First  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        506  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        10  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         Best First With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        393859  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Manhattan Distance)  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        11  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  
  
Function Name:         A* With Heuristic (Out of Place)  
Final State:           123456789ABCDEF0  
Action Set:            uldllurrdr  
Path Cost:             10  
Nodes Expanded:        2307  
Original State:        12345678D9CFEBA0  
Verification of path:  123456789ABCDEF0  


#**Assignment Problems 4x4 Hard**

In [ ]:
listOf4x4HProblems = ["71A92CE03DB4658F", "02348697DF5A1EBC", "39A1D0EC7BF86452", "EAB480FC19D56237", "7DB13C52F46E80A9"]

BoardSize = 4
    
GoalState3x3 = list("123456780")
GoalState4x4 = list("123456789ABCDEF0")
       
GoalState = (GoalState3x3, GoalState4x4)[BoardSize == 4]

for problemSet in listOf4x4HProblems:
  problem = Problem(list(problemSet), GoalState,  BoardSize)

  nodeList1 = [ (best_first_search(problem), "Best First w Path Cost")  
              # (breadth_first_search(problem), "Breadth First"), 
              # #(depth_first_search(problem), "Depth First"), 
              # (iterative_deepening_search(problem, 30), "Iterative Deepening Depth First")
            ]

  for ((node, expandedNodes), functionName) in nodeList1:
    printNodeVals(node, expandedNodes, functionName, '' + problemSet)

  # nodeList2 = [(best_first_search_w_heuristic(problem, getManhattanDis), "Best First With Heuristic (Manhattan Distance)" ),
  #             (best_first_search_w_heuristic(problem, getOutOfPlace), "Best First With Heuristic (Out of Place)") ,
  #             (a_star_search(problem, getManhattanDis), "A* With Heuristic (Manhattan Distance)" ),
  #             (a_star_search(problem, getOutOfPlace),"A* With Heuristic (Out of Place)") ]
  # for ((node, expandedNodes), functionName) in nodeList2:
  #   printNodeVals(node, expandedNodes, functionName, '' + problemSet)

KeyboardInterrupt: ignored